# **RED NEURONAL CIFRADO CESAR. BÚSQUEDA DE PARÁMETROS ÓPTIMOS**

In [ ]:
from keras.models import Sequential,Model
from keras.layers import SimpleRNN, Input, Dense
import numpy as np
import matplotlib.pyplot as plt 
from keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import string
import random

# Parametros configuración inicial
word_size = 3 # tamaño cadenas
shift = 3 # Desplazamiento letra (cifrado)

# Declaración de parámetros
nn = [120,80,120,80,26*word_size]
X_train = []
Y_train = []
X_test = []
Y_test = []
X = []
Y = []

# Generar el dataset de entrenamiento
X, Y = generate_words(10000,0,X)
X_train = wordToNumber(X)
Y_train = wordToNumber(Y)

# Generar el dataset de test (datos nuevos, no usados en el entrenamiento)
X, Y = generate_words(2000,1,X)
X_test = wordToNumber(X)
Y_test = wordToNumber(Y)

In [ ]:
def crearModelo(loss='poisson',activation='relu', unit1=120, unit2=80,optimizer='RMSprop'):

  # Definir modelo    
  model = Sequential()

  # Capas
  model.add(Dense(nn[4], input_dim=(26*word_size)))
  model.add(Dense(unit1, activation))
  model.add(Dense(unit2, activation))
  model.add(Dense(unit1, activation))
  model.add(Dense(nn[4], activation))

  # Compilar modelo
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

In [ ]:
batch_size = [200,300,500]
epochs = [50, 100, 150]
optimizer = ['SGD', 'RMSprop', 'Adagrad','Adam', 'Adamax', 'Nadam']
loss = ['binary_crossentropy','categorical_crossentropy','poisson']
activation = ['softmax','relu','sigmoid','softplus']
unit1 = [80,100,120]
unit2 = [100]

In [ ]:
model = KerasClassifier(build_fn=crearModelo, epochs=100, batch_size=10, verbose=0)

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, unit1=unit1, unit2=unit2, loss=loss, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
def generate_words (number_words,test,X_train):
  words = []

  if(test == 0):
    for x in range(number_words):
      words.append(''.join(random.SystemRandom().choice(string.ascii_letters).lower() for _ in range(word_size)))
  else:
    for x in range(number_words):
      repeated = 1
      while(repeated == 1):
        word = (''.join(random.SystemRandom().choice(string.ascii_letters).lower() for _ in range(word_size)))
        if(word not in X_train):
          words.append(word)
          repeated = 0

  encrypted_words = []

  for word in words:
    cadena = ""
    for x in range(0,word_size):
      cadena += chr((((ord(word[x]) - 97) + shift) % 26) + 97)
    encrypted_words.append(cadena)

  return encrypted_words, words

In [ ]:
def wordToNumber(words):

    word_id = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9,'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14,'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19,'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
    lista = []
    x = 0
    one_hot_encoding = np.zeros((len(words),len(word_id)*word_size))
    for word in words:  
      for i in range(0,word_size):
        one_hot_encoding[x][word_id[word[i]]+(26*i)] = 1
      x+=1
    return one_hot_encoding
